<a href="https://colab.research.google.com/github/nkiru-ede/nkiru_codes/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing important packages
pip install twython

In [ ]:
pip install nltk

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [ ]:
#importing important libraries
import pandas as pd
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from datetime import date
from datetime import time
from datetime import datetime
import seaborn as sns
import matplotlib.font_manager
from itertools import cycle, islice
import numpy as np
import tweepy
import requests
import base64
import nltk
from nltk.corpus import stopwords
from nltk import corpus
from geopy.geocoders import Nominatim
import numpy as np
import gmplot
import webbrowser
from twython import Twython
import json
from twython import Twython
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.ticker as ticker

In [ ]:
#Get Twitter credentials into Json format
credentials={}
credentials['CONSUMER_KEY'] = 'k45uvcP2VAYtiMcv5WPZmf6wC'
credentials['CONSUMER_SECRET'] = 'g2KxMpvOBj5ebQpExDVFAuYHrkTU99ICAcdNBprbTHieS6US0n'
credentials['ACCESS_TOKEN'] = '3102441035-Wdz2E2JPzu9Piom64ZAQ9rJZ3JbOYNkEHJAwM2C'
credentials['ACCESS_SECRET'] = 'GyFKlqD6hgfjZGsN6j3YMq3peEcPw96I0FyrM5Mfvqn1y'

with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [ ]:
#Streaming data for sentiment analysis

from twython import TwythonStreamer
import csv
import json

with open('twitter_credentials.json','r') as file:
    creds = json.load(file)

def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user_screenName'] = tweet['user']['screen_name']
    d['user_name'] = tweet['user']['name']
    d['user_loc'] = tweet['user']['location']
    d['source'] = tweet['source']
    d['verified'] = tweet['user']['verified']
    d['created_at'] = tweet['created_at']
    d['followers'] = tweet['user']['followers_count']
    d['retweet'] = tweet['retweet_count']
    d['coordinates'] = tweet['coordinates']
    return d

#Create class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):
    #recieved data
    def on_success(self,data):
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)
            
    def on_error(self,status_code, data):
        print(status_code, data)
        self.disconnect()
        
    def save_to_csv(self,tweet):
        with open (r'NeilParish.csv','a', encoding = 'utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))
#load credentials
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
stream.statuses.filter(track='Neil Parish')


In [ ]:
#add column headers to dataframe
df = pd.read_csv("NeilParish.csv",
                  names=["hashtags", "text", "user_screenName", "user_name","user_loc", "source", "verified", "created_at", "followers", "retweet", "coordinates"])
#print new dataframe
df

In [ ]:
#extract only tweet text from dataframe
tweet =  df.text
df = pd.DataFrame(tweet)

In [ ]:
#Clean text - sentiment analysis
def cleantext(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #removes @mentions
  text = re.sub(r'#','', text) # removes #
  text = re.sub(r'RT[\s]+', '', text)#removes RT
  text = re.sub(r'https?:/\/\S+', '', text)#removes hhtp
  return text
df['text'] = df['text'].apply(cleantext)
#print new dataframe
df

In [ ]:
#Create a function to get the subjectivity
#subjectivity - tells how subjective the text/opinion is
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
#polarity tells how positive or negative a text is
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
 
#create two new columns
df['Subjectivity'] = df['text'].apply(getSubjectivity)
df['Polarity'] = df['text'].apply(getPolarity)

#show the new dataframe
df

In [ ]:
#Create a function to compute the negative, neautral and positive analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'
df['Analysis'] = df['Polarity'].apply(getAnalysis)
df

In [ ]:
#print the negative tweets
j=1
sortedDF = df.sort_values(by=['Polarity'], ascending = 'False')
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Negative'):
    print(str(j)+')'+sortedDF['text'][i])
    print()
    j=j+1

In [ ]:
#print the positive tweets
j=1
sortedDF = df.sort_values(by=['Polarity'], ascending = 'False')
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Positive'):
    print(str(j)+')'+sortedDF['text'][i])
    print()
    j=j+1

In [ ]:
#print the neutral tweets
j=1
sortedDF = df.sort_values(by=['Polarity'], ascending = 'False')
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Neutral'):
    print(str(j)+')'+sortedDF['text'][i])
    print()
    j=j+1

In [ ]:
#put the different sentiments in seperate dataframes
Positive = df[df['Analysis']== "Positive"]
Negative = df[df['Analysis']== "Negative"]
Neutral = df[df['Analysis']== "Neutral"]

In [ ]:
#Function to visualise dataframe - WordCloud and Frequency Distribution
def visualiser(data):
    fdist = nltk.FreqDist()
    en_stop = set(stopwords.words('english'))
    for i in data["text"]:
      i = re.sub(r"[^a-zA-Z0-9]+"," ", i)
      i = nltk.word_tokenize(i)
      stopped_tokens = [t for t in i if not t in en_stop]
      for j in stopped_tokens:
        fdist[j] +=1
    fdist.plot(30, cumulative=False)
    
    wordcloud = WordCloud(max_font_size=50, max_words = 100, background_color="white").generate_from_frequencies(fdist)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
#Visualise positive sentiment tweets
visualiser(Positive)

In [ ]:
#Visualise negative sentiment tweets
visualiser(Negative)

In [ ]:
#Visualise neutral sentiment tweets
visualiser(Neutral)

In [ ]:
#scatter plot the different sentiments
sns.set(style="darkgrid")    
fig, ax = plt.subplots(figsize=(8, 5))    
palette = sns.color_palette("bright", 3)
g = sns.scatterplot(ax=ax, x="Polarity", y="Subjectivity", hue="Analysis", marker='o', data=df, s=100, palette= palette)
g.legend(bbox_to_anchor=(1, 1), ncol=1)
#g.set(xlim = (50000,250000))

In [ ]:
#get the percentage of positive tweets
ptweets = df[df.Analysis=='Positive']
ptweets = ptweets['text']
ptweets
round( (ptweets.shape[0] /df.shape[0]) * 100 , 1)

In [ ]:
#get the percentage of negative tweets
ntweets = df[df.Analysis=='Negative']
ntweets = ntweets['text']
ntweets
round( (ntweets.shape[0] /df.shape[0]) * 100 , 1)

In [ ]:
#get the percentage of neutral tweets
nntweets = df[df.Analysis=='Neutral']
nntweets = nntweets['text']
nntweets
round( (nntweets.shape[0] /df.shape[0]) * 100 , 1)

In [ ]:
#show the value count
df['Analysis'].value_counts()
#plot and visualize the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar', color = list('gcrymc'))

plt.show()